In [9]:
def create_best_lstm_model(best_params):
    model = Sequential()
    n_layers = best_params['lstm_n_layers']
    units = best_params['lstm_units']
    for i in range(n_layers):
        model.add(LSTM(
            units=units,
            return_sequences=(i < n_layers - 1),
            input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]) if i == 0 else None
        ))
        model.add(Dropout(rate=best_params['lstm_dropout']))
    model.add(Dense(1))
    learning_rate = best_params['lstm_learning_rate']
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

def create_lstm_model(trial):
    model = Sequential()
    n_layers = trial.suggest_int('lstm_n_layers', 2, 4)  # Optimize number of LSTM layers
    units = trial.suggest_categorical('lstm_units', [50, 100, 150, 200])
    for i in range(n_layers):
        model.add(LSTM(
            units=units,
            return_sequences=(i < n_layers - 1),
            input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]) if i == 0 else None
        ))
        dropout_rate = trial.suggest_categorical('lstm_dropout', [0.1, 0.2, 0.3, 0.4, 0.5])
        model.add(Dropout(rate=dropout_rate))  # Use specific dropout rate
    model.add(Dense(1))
    learning_rate = trial.suggest_categorical('lstm_learning_rate', [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001])  # Optimize learning rate
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# Define optimization objective function
def objective_LSTM(trial):
    num_samples = X_train_lstm.shape[0] * 0.8
    batch_size_candidates = [16, 32, 64, 128, 256, 512, 1024, 2048]
    batch_size_candidates = [bs for bs in batch_size_candidates if bs <= num_samples]
    param = {
        'epochs': trial.suggest_categorical('lstm_epochs', [10, 20, 30, 40, 50, 75, 100, 150, 200]),
        'batch_size': trial.suggest_categorical('lstm_batch_size', batch_size_candidates)
    }
    param['batch_size'] = min(param['batch_size'], num_samples)  # Ensure batch_size does not exceed the number of samples in the training dataset
    model = KerasRegressor(build_fn=lambda: create_lstm_model(trial), epochs=param['epochs'], batch_size=param['batch_size'], verbose=0)
    
    # Use 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mse_scores = []
    
    for train_index, val_index in kf.split(X_train_lstm):
        X_train, X_val = X_train_lstm[train_index], X_train_lstm[val_index]
        y_train_fold, y_val = y_train[train_index], y_train[val_index]
        num_samples = X_train.shape[0]
        # 'patience' parameter defines how many epochs the model continues training without improvement before stopping
        # 'restore_best_weights' if True, the model's weights will be restored to the point with the lowest validation loss
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        model.fit(X_train, y_train_fold, validation_data=(X_val, y_val), callbacks=[early_stopping, KerasPruningCallback(trial, 'val_loss')])
        
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        mse_scores.append(mse)
    
    return np.mean(mse_scores)

def plt_scatter(data, filename):
    dataframes = data[['predicts', 'records']]
    dataframes1 = dataframes
    r2 = r2_score(dataframes['records'], dataframes['predicts'])
    # Calculate NRMSE
    mse = mean_squared_error(dataframes['records'], dataframes['predicts'])
    nrmse = np.sqrt(mse) / np.mean(dataframes['records'])
    rrmse = calculate_rrmse1(dataframes['records'], dataframes['predicts'])
    
    mape = mean_absolute_percentage_error(dataframes['records'], dataframes['predicts'])
    acc = calculate_acc(dataframes['records'], dataframes['predicts'])
    plt.figure(figsize=(10, 6))
    plt.hexbin(dataframes['records'], dataframes['predicts'], gridsize=50, cmap='viridis', mincnt=1)
    cb = plt.colorbar(label='Density')
    fit_params = np.polyfit(dataframes['records'], dataframes['predicts'], 1)
    fit_line = np.polyval(fit_params, dataframes['records'])
    plt.plot(dataframes['records'], fit_line, color='red', label='Fit line')  # Add fitting line
    max_val = max(max(dataframes['records']), max(dataframes['predicts']))
    plt.plot([0, max_val], [0, max_val], linestyle='--', color='green', label='1:1 line')
    plt.title('')
    plt.xlabel('Observed values')
    plt.ylabel('Predicted values')
    plt.grid(True)
    plt.text(0.02, 0.95, f'R² = {r2:.2f}', transform=plt.gca().transAxes, fontsize=12, va='top')
    plt.text(0.02, 0.90, f'ACC = {acc:.2f}', transform=plt.gca().transAxes, fontsize=12, va='top')
    plt.text(0.02, 0.85, f'RRMSE = {rrmse:.2f}%', transform=plt.gca().transAxes, fontsize=12, va='top')
    plt.text(0.02, 0.80, f'MAPE = {mape*100:.2f}%', transform=plt.gca().transAxes, fontsize=12, va='top')
    
    # Display and save the figure
    plt.savefig(filename)
    plt.show()

def extract_selected_variables(inputpath_base):
    inpath_dates = os.path.join(inputpath_base, 'dataset', "01_inputdata", 'selectFeatures.txt')
    # Construct file path
    gs_infornamtion = pd.read_csv(inpath_dates, sep='\t', header=None)
    gs_infornamtion.columns = ['slected_dynamic_features', 'slected_static', 'regionID']
    gs_infornamtion['slected_dynamic_features'] = gs_infornamtion['slected_dynamic_features'].apply(ast.literal_eval)
    gs_infornamtion['slected_static'] = gs_infornamtion['slected_static'].apply(ast.literal_eval)
    return gs_infornamtion

def calculate_rrmse1(y_true, y_pred):
    """
    Calculate RRMSE (Relative Root Mean Square Error) using the mean of actual values as reference.
    
    Parameters:
    y_true -- Array or list of true values
    y_pred -- Array or list of predicted values
    
    Returns:
    rrmse -- Relative Root Mean Square Error (percentage)
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate RMSE
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    
    # Calculate mean of true values
    mean_y_true = np.mean(y_true)
    
    # Calculate RRMSE
    rrmse = (rmse / mean_y_true) * 100
    
    return rrmse

def calculate_rrmse2(y_true, y_pred):
    """
    Calculate rRMSE (Relative Root Mean Square Error) using each individual actual value as reference.
    
    Parameters:
    y_true -- Array or list of true values
    y_pred -- Array or list of predicted values
    
    Returns:
    rrmse -- Relative Root Mean Square Error (percentage)
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate rRMSE
    rrmse = np.sqrt(np.mean(((y_true - y_pred) / y_true) ** 2)) * 100
    
    return rrmse

# Define custom nRMSE evaluation function
def calculate_nrmse(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    nrmse = rmse / (y_true.max() - y_true.min())
    return nrmse * 100

def calculate_acc(y_true, y_pred):
    # Calculate means of observed and predicted values
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    
    # Calculate anomalies (deviations from mean)
    anomaly_true = y_true - mean_true
    anomaly_pred = y_pred - mean_pred
    
    # Calculate ACC (Anomaly Correlation Coefficient)
    numerator = np.sum(anomaly_true * anomaly_pred)
    denominator = np.sqrt(np.sum(anomaly_true**2) * np.sum(anomaly_pred**2))
    
    acc = numerator / denominator
    return acc

In [18]:
import os
sys.path.append(r'C:\ProgramData\anaconda3\Lib\site-packages') 
sys.path.append(r'C:\Users\DELL\.conda\envs\myenv\Lib\site-packages') 
sys.path.append(r'C:\Users\DELL\.conda\envs\rasterio_env\Lib\site-packages') 
import sys
import ast
import pandas as pd
import numpy as np
import optuna
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from optuna_integration.keras import KerasPruningCallback
from optuna.samplers import TPESampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
# from tqdm.notebook import tqdm
import tensorflow.keras.backend as K
import json

In [13]:
'''
# Required Packages to Install
Pre-installed Packages:
os, sys, warnings, optuna 

Packages to Install:
matplotlib, sklearn, optuna_integration, tensorflow, scikeras, ast

Installation Commands:
conda install optuna matplotlib sklearn optuna_integration tensorflow scikeras ast -c pytorch
conda install matplotlib tensorflow scikeras -c pytorch
'''

countryID = '06_India';  # Country identifier (format: XX_CountryName)
country = countryID[3:];  # Country name extracted from countryID
crop = '02_wheat';  # Crop type (format: XX_CropName)
yield_type = 'actual_yield';  # Yield type to predict (e.g., actual yield, potential yield)

inputpath_base = os.path.dirname(os.getcwd())  # Base input path (parent directory of current working directory)
startyear = 2001;  # Start year of the dataset
endyear = 2018;    # End year of the dataset
Forecastyear = endyear;  # Forecast target year (same as endyear in this configuration)
region = 'I';  # Region identifier (custom region code)
Forecastyears = {
    'I': Forecastyear  # Forecast years corresponding to each region (key: region ID, value: forecast year)
}
years = range(startyear, endyear + 1)  # Range of years covered in the dataset
years_str = [str(year) for year in years]  # String format of dataset years (for file naming/path construction)
modelname = 'LSTM';  # Machine learning model name (here: Long Short-Term Memory network)
yield_type = 'actual_yield';  # Reconfirm yield type (consistent with earlier definition)
SelFeature_infornamtion = extract_selected_variables(inputpath_base)  # Extract selected feature information (dynamic and static features)
institution = 'ECMWF';  # Meteorological data provider (European Centre for Medium-Range Weather Forecasts)

# Read selected growth stage week information
inpath_dates = os.path.join(inputpath_base, 'dataset', "01_inputdata", 'gs_three_periods.txt')  # Path to growth stage date file
gs_infornamtion = pd.read_csv(inpath_dates, delim_whitespace=True, header=None)  # Read growth stage data (tab-separated)
gs_infornamtion.columns = ['start_point', 'peak', 'harvest_point', 'VI_select2', 'regionID']  # Rename columns: start point, peak stage, harvest point, selected VI, region ID
# Extract growth stage parameters for the target region
start_point, peak, harvest_point, VI_select2, region = gs_infornamtion[gs_infornamtion['regionID'] == region].iloc[0]

In [ ]:
###########Hyperparameter Tuning###########################################
###################Load Selected Feature Variables#########################################
Forecastyear = Forecastyears[region]  # Get forecast year for the target region
# Load original dataset
data = pd.read_csv(os.path.join(inputpath_base, 'dataset', "01_inputdata", region + '_data_ori.csv'))
data = data.drop_duplicates(subset=['year', 'idJoin'], keep='first')  # Remove duplicate records (retain first occurrence)
data = data[data['year'] != Forecastyear]  # Exclude data from the forecast year

# Extract selected dynamic features, static features, and region ID
TimeFeatures_sel, Static_sel, regionID = SelFeature_infornamtion[SelFeature_infornamtion['regionID'] == region].iloc[0]
feature_all = TimeFeatures_sel + Static_sel  # Combine dynamic and static features into a complete feature list
# Filter columns containing any of the selected features
filtered_columns = [col for col in data.columns if any(feature in col for feature in feature_all)]
filtered_columns = [col for col in filtered_columns if 'year.1' not in col]  # Exclude redundant year column ('year.1')

####Exclude yield-related features (for correlation testing)#########################
filtered_columns = [col for col in filtered_columns if 'Yield' not in col]  # Exclude yield-related features
Static_sel = [col for col in Static_sel if 'Yield' not in col]  # Update static features (remove yield-related items)

# Construct machine learning dataset (selected features + target yield)
MLdata_reduced = data[filtered_columns + [yield_type]]
MLdata_reduced['year'] = data['year']  # Add year column back to the dataset


############################Data Standardization###########################################################
data_all = MLdata_reduced  # Full dataset for modeling
X_all = data_all.drop([yield_type], axis=1)  # Feature matrix (exclude target variable)
y_all = data_all[yield_type]  # Target variable (yield)

# Standardize feature matrix (Z-score normalization: mean=0, std=1)
scaler_X = StandardScaler().fit(X_all)
X = scaler_X.transform(X_all)
# Standardize target variable
scaler_y = StandardScaler().fit(y_all.values.reshape(-1, 1))
y = scaler_y.transform(y_all.values.reshape(-1, 1)).flatten()

# Convert standardized features back to DataFrame (for easier column-based operations)
X = pd.DataFrame(data=X, columns=X_all.columns.tolist())

# Generate list of growth stage weeks (handle cross-year growth periods)
if start_point > harvest_point:
    # If start week > harvest week (cross-year growth, e.g., winter crops), combine two ranges
    weeks_select_list = list(range(start_point, 47)) + list(range(1, harvest_point + 1))
else:
    # Normal growth period (same year), direct range from start to harvest week
    weeks_select_list = list(range(start_point, harvest_point + 1))

# Reshape data into 3D format (samples, time steps, features) required for LSTM
data_list = []
for i in weeks_select_list:
    # Extract features for the i-th week (dynamic features + static features)
    data_i = X[[f'Week{i}_{feature}' for feature in TimeFeatures_sel] + Static_sel]
    data_list.append(data_i.values)  # Store weekly feature matrices in a list

# Stack weekly feature matrices into a 3D array
data_list = np.array(data_list)
# Reshape to (samples, time steps, features) format (LSTM input requirement)
X_train_lstm = np.transpose(data_list, (1, 0, 2))
y_train = y  # Target variable (standardized yield)

##################Create LSTM Model and Perform Hyperparameter Tuning####################################
# Define storage path for tuning results (SQLite database, stored in the code directory)
storage_name = f"sqlite:///{country}_{modelname}_region{region}.db"
study_name = f"{country}_{modelname}_region{region}"  # Name of the Optuna study

# Create Optuna study: minimize loss, use TPE sampler, load existing study if it exists
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(),
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True
)

# Run hyperparameter optimization: 200 trials, 4 parallel jobs
study.optimize(objective_LSTM, n_trials=200, n_jobs=4)

# Output best hyperparameters and corresponding minimum loss
print('Best parameters: ', study.best_params)
print('Best score (mean MSE): ', study.best_value)

# Save best hyperparameters to JSON file
save_dir = os.path.join(inputpath_base, 'best_params')
os.makedirs(save_dir, exist_ok=True)  # Create directory if it doesn't exist
save_path = os.path.join(save_dir, f"{study_name}_best_params.json")
with open(save_path, "w") as f:
    json.dump(study.best_params, f, indent=4)  # Save with indentation for readability
print("Best hyperparameters have been saved to JSON file")